# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

### Importando bibliotecas

In [13]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [14]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd
import plotly.express as px

### Criando sessão Spark

In [15]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [16]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [17]:
spark

### Importanto dados tratados da view do arquivo CSV

In [18]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/dados_uteis_gz/2023-11-01_pnad_covid_view.csv.gz', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

23/11/02 08:46:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

### Análise exploratória de características gerais

In [19]:
df_temp = df.createOrReplaceTempView('df_temp') #criando view temporária na sessão Spark SQL

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença em relação ao sexo, raça e escolaridade</p>**

**Qual é a porcentagem de infectados por sexo?**

Verifica-se que houve mais contaminação no grupo de mulheres testadas

df_testes_validos_sexo = spark.sql(
    '''
        SELECT sexo, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY sexo
    '''
).toPandas()

df_testes_positivos_sexo = spark.sql(
    '''
        SELECT sexo, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY sexo
    '''
).toPandas()

df_infectados_sexo = pd.merge(df_testes_validos_sexo, df_testes_positivos_sexo, on='sexo')
df_infectados_sexo['percentual_testes_positivos'] = ((df_infectados_sexo.qtd_testes_positivos / df_infectados_sexo.qtd_testes_validos) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_infectados_sexo.sort_values('percentual_testes_positivos', ascending=False), 
    values = 'percentual_testes_positivos', 
    names = 'sexo',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos':'Percentual de testes positivos',
        'sexo':'Sexo'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por sexo</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcentagem de infectados por cor/raça?**

Indígenas obtiveram, proporcionalmente ao número de indivíduos testados, maior número de casos confirmados

df_testes_validos_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND cor_raca != 'Ignorado'
        GROUP BY cor_raca

    '''
).toPandas()


df_testes_positivos_raca = spark.sql(
    '''
        SELECT cor_raca, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND cor_raca != 'Ignorado'
        GROUP BY cor_raca
    '''
).toPandas()

df_infectados_cor_raca = pd.merge(df_testes_validos_cor_raca, df_testes_positivos_raca, on='cor_raca')
df_infectados_cor_raca['percentual_testes_positivos'] = ((df_infectados_cor_raca.qtd_testes_positivos / df_infectados_cor_raca.qtd_testes_validos) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_infectados_cor_raca.sort_values('percentual_testes_positivos', ascending=False), 
    values = 'percentual_testes_positivos', 
    names = 'cor_raca',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'cor_raca': 'Cor/raça'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por cor/raça</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcentagem de infectados por escolaridade?**

df_testes_validos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY escolaridade
    '''
).toPandas()

df_testes_positivos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY escolaridade
    '''
).toPandas()

df_infectados_escolaridade = pd.merge(df_testes_validos_escolaridade, df_testes_positivos_escolaridade, on='escolaridade')
df_infectados_escolaridade['percentual_testes_positivos'] = ((df_infectados_escolaridade.qtd_testes_positivos / df_infectados_escolaridade.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_infectados_escolaridade.sort_values('percentual_testes_positivos', ascending=False),
    x='escolaridade',
    y='percentual_testes_positivos',
    color='escolaridade',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'escolaridade': 'Escolaridade'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por escolaridade</b>',
    yaxis_title='%',
    showlegend=False,
    width=800,  
    height=600
)

fig.update_xaxes(tickangle=45)

fig.show()

#### **<p style='color:gray'> Estatísticas geoespaciais da incidência da doença</p>**

**Há diferença significativa de incidência da doença por zona de domicílio?**

Não foi constatada diferença significativa no número de infectados por zona de domicílio.

df_testes_validos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY situacao_domicilio
    '''
).toPandas()

df_positivos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
    '''
).toPandas()


df_infectados_zona = pd.merge(df_testes_validos_zona, df_positivos_zona, on='situacao_domicilio')
df_infectados_zona['percentual_testes_positivos'] = ((df_infectados_zona.qtd_testes_positivos / df_infectados_zona.qtd_testes_validos) * 100).round(decimals=2)

fig = px.pie(
    data_frame=df_infectados_zona.sort_values('percentual_testes_positivos', ascending=False), 
    values = 'percentual_testes_positivos', 
    names = 'situacao_domicilio',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos':'Percentual de testes positivos',
        'situacao_domicilio':'Zona de domicílio'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por zona de domicílio</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a quantidade de infectados por estado?**

df_cotagio_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
    '''
).toPandas()

fig = px.bar(
    data_frame=df_cotagio_estado.sort_values('qtd_testes_positivos', ascending=False),
    x='uf',
    y='qtd_testes_positivos',
    color='uf',
    color_discrete_sequence=['#8B0000'],
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'uf': 'Estado'
    }
)

fig.update_layout(
    title='<b>Quantidade de infectados por estado</b>',
    yaxis_title='Quantidade de infectados',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

**Qual é quantidade de infectados por região?**

df_cotagio_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) as qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()

fig = px.bar(
    data_frame=df_cotagio_regiao.sort_values('qtd_testes_positivos', ascending=False),
    x='regiao',
    y='qtd_testes_positivos',
    color='regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade de infectados',
        'regiao': 'Região'
    }
)

fig.update_layout(
    title='<b>Quantidade de infectados por região</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a taxa de incidência por estado?**

Observa-se que a taxa é tanto maior quanto mais ao norte do país

df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT uf, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf
    '''
).toPandas()

df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
    '''
).toPandas()

df_taxa_de_contagio_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='uf')
df_taxa_de_contagio_estado['taxa_de_contagio_mil_habitantes'] = ((df_taxa_de_contagio_estado['qtd_testes_positivos'] / df_taxa_de_contagio_estado['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_de_contagio_estado.sort_values('taxa_de_contagio_mil_habitantes', ascending=False),
    x='uf',
    y='taxa_de_contagio_mil_habitantes',
    color='uf',
    color_discrete_sequence=['#8B0000'],
    labels={
        'taxa_de_contagio_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'uf': 'Estado'
    }
)

fig.update_layout(
    title='<b>Taxa de incidência por estado (por mil habitantes)</b>',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

**Qual é a taxa de incidência por região?**

df_qtd_testes_validos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
    '''
).toPandas()


df_qtd_testes_positivos_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()

df_taxa_de_contagio_regiao = pd.merge(df_qtd_testes_validos_regiao, df_qtd_testes_positivos_regiao, on='regiao')
df_taxa_de_contagio_regiao['taxa_de_contagio_mil_habitantes'] = ((df_taxa_de_contagio_regiao['qtd_testes_positivos'] / df_taxa_de_contagio_regiao['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_de_contagio_regiao.sort_values('taxa_de_contagio_mil_habitantes', ascending=False),
    x='regiao',
    y='taxa_de_contagio_mil_habitantes',
    color='regiao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_de_contagio_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'regiao': 'Região'
    }
)

fig.update_layout(
    title='<b>Taxa de incidência por região (por mil habitantes)</b>',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

**Mapeamento de risco de acordo com a taxa de contagio por mil habitantes**

import json

geojson = json.load(open('dados/dados_importados/brasil_estados.json'))

df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT uf, sigla, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf, sigla
    '''
).toPandas()

df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT sigla, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY sigla
    '''
).toPandas()

df_taxa_de_contagio_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='sigla')
df_taxa_de_contagio_estado['taxa_de_contagio_mil_habitantes'] = ((df_taxa_de_contagio_estado['qtd_testes_positivos'] / df_taxa_de_contagio_estado['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.choropleth(
    data_frame=df_taxa_de_contagio_estado,
    geojson=geojson,
    locations='sigla',
    color='taxa_de_contagio_mil_habitantes',
    hover_name='sigla',
    scope='south america',
    color_continuous_scale=px.colors.sequential.Reds,
    labels={
        'taxa_de_contagio_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'sigla': 'Estado'
    }
)

fig.add_scattergeo(
    geojson=geojson,
    locations=df_taxa_de_contagio_estado['sigla'],
    text=df_taxa_de_contagio_estado['sigla'],
    textposition='middle center',
    mode='text',
    textfont=dict(
        size=14,
        color='black',
        
    )
)

fig.update_layout(
    title='<b>Mapa de risco de acordo com a taxa de incidência nos estados</b>',
    autosize=False,
    margin = dict(
        l=0,
        r=0,
        b=0,
        autoexpand=True
    ),
    width=1600, 
    height=1000
)

fig.show()

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença em relação à faixa etária do esquema vacinal</p>**

**Qual é a taxa de incidência por faixa etária do esquema vacinal nos estados?**

O planejamento da vacinação para 2023 tem como proposta de esquema o uso de vacinas bivalentes com cepas atualizadas com dose de reforço (booster) para grupos específicos em maior vulnerabilidade, riscos para complicação e óbito e maior exposição, e o uso de vacinas monovalentes para dar início ou completar esquema vacinal das pessoas que não fazem parte dos grupos prioritários elegíveis para vacinação bivalente. Vale ressaltar que ambas as vacinas, monovalentes e bivalentes, agem do mesmo modo no organismo, estimulando o sistema imunológico a produzir anticorpos protetores e células de defesa contra o vírus SARS-CoV-2.

Quando infectada pelo vírus, a pessoa vacinada conseguirá combatê-lo rapidamente, pois já tem imunidade contra ele e assim evoluirá de modo assintomático ou com doença leve (maioria dos casos). O que difere a vacina monovalente da vacina bivalente é a capacidade das vacinas bivalentes de estimularem uma resposta imune mais efetiva contra a variante Ômicron, garantindo assim maior proteção contra a infecção pelo SARS-CoV-2. Cabe ressaltar, que as vacinas monovalentes continuam a proteger as pessoas completamente vacinadas contra hospitalizações, SRAG e óbito. As recomendações atuais para imunização contra a covid-19 no país são estabelecidas de acordo com as faixas etárias, os imunizantes disponíveis, as recomendações dos fabricantes e os resultados de estudos nacionais e internacionais. 

6 meses a 4 anos, 11 meses e 29 dias:

Para crianças que iniciaram o esquema vacinal recomendado para a faixa etária de 6 meses a 4 anos, 11 meses e 29 dias com a vacina Pfizer (frasco de tampa vinho) e finalizarão seu esquema após completar 5 anos de idade, essas deverão adotar o esquema recomendado para a faixa etária de 5 a 11 anos utilizando a vacina Pfizer (frasco de tampa laranja) para completar o seu esquema.


3 e 4 anos, 11 meses e 29 dias:

Para crianças de 3 e 4 anos, 11 meses e 29 dias que iniciaram o esquema vacinal com CoronaVac, o esquema primário é composto por duas doses (1ª Dose + 2ª Dose), com intervalo de quatro semanas entre a primeira e segunda doses. Nessa faixa etária há a indicação de uma dose de reforço preferencialmente com a vacina da Pfizer (frasco de tampa vinho), com intervalo de quatro meses após a segunda dose recebida. Na indisponibilidade dessa o reforço poderá ser realizado com a vacina CoronaVac. O registro da dose de reforço com a vacina CoronaVac no sistema de informação deverá ser 1º reforço (R1) e com a vacina Pfizer 3ª dose (D3). 

5 a 11 anos de idade:

Para crianças de 5 a 11 anos de idade, o esquema primário recomendado é composto por duas doses da vacina Covid-19 (1ª Dose + 2ª Dose). Para aquelas que iniciaram o esquema com o imunizante CoronaVac, o intervalo entre as doses é de quatro semanas, e, para as crianças que iniciaram o esquema com o imunizante Covid-19 Pfizer (frasco de tampa laranja), o intervalo é de oito semanas após a 1ª dose. A dose de reforço deve ser feita com intervalo mínimo de quatro meses após a segunda dose.

12 a 39 anos de idade:

Para as pessoas de 12 a 39 anos de idade não incluídas no grupo prioritário recomendado para receber as vacinas bivalentes e que não iniciaram a vacinação ou que estão com esquema vacinal incompleto, deverá ser realizado o esquema primário utilizando duas doses das vacinas Covid-19 (monovalente) e o reforço com intervalo mínimo de 4 meses entre as doses.

40 a 59 anos de idade:

Para adultos de 40 a 59 anos de idade não incluídos no grupo prioritário recomendado para receber as vacinas bivalentes, o esquema vacinal é composto por duas doses (1ª Dose + 2ª Dose) e duas doses de reforço (1º Reforço + 2º Reforço).

df_qtd_testes_validos_faixa_etaria = spark.sql(
    '''
        SELECT
                (CASE
                        WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                        WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                        WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                        WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                        WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                        WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY faixa_etaria
    '''
).toPandas()

df_qtd_testes_positivos_faixa_etaria = spark.sql(
    '''
        SELECT
                (CASE
                    WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                    WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                    WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                    WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                    WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                    WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_etaria
    '''
).toPandas()

df_taxa_cotagio_faixa_etaria = pd.merge(df_qtd_testes_validos_faixa_etaria, df_qtd_testes_positivos_faixa_etaria, on='faixa_etaria')
df_taxa_cotagio_faixa_etaria['taxa_de_contagio_mil_habitantes']= ((df_taxa_cotagio_faixa_etaria['qtd_testes_positivos'] / df_taxa_cotagio_faixa_etaria['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_cotagio_faixa_etaria.sort_values('taxa_de_contagio_mil_habitantes', ascending=False),
    x='faixa_etaria',
    y='taxa_de_contagio_mil_habitantes',
    color='faixa_etaria',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'taxa_de_contagio_mil_habitantes': 'Taxa de incidência (por mil habitantes)',
        'faixa_etaria': 'Faixa etária'
    }
)

fig.update_layout(
    title='<b>Taxa de incidência nos estados por faixa etária do esquema vacinal (por mil habitantes)</b>',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### **<p style='color:gray'> Estatísticas descritivas da incidência da doença em relação ao rendimento</p>**

**Qual é a quantidade de infectados por faixa de rendimento?**

df_faixa_rendimento_infectados = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_rendimento
    '''
).toPandas()

fig = px.bar(
    data_frame=df_faixa_rendimento_infectados.sort_values('qtd_testes_positivos', ascending=False),
    x='faixa_rendimento',
    y='qtd_testes_positivos', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos': 'Quantidade',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)

fig.update_layout(
    title='<b>Quantidade de infectados por faixa de rendimento</b>',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

**Qual é a porcentagem de infectados por faixa de rendimento?**

df_testes_validos_faixa_rendimento = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()

df_testes_positivos_faixa_rendimento = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()

df_percentual_cotagio_faixa_rendimento = pd.merge(df_testes_validos_faixa_rendimento, df_testes_positivos_faixa_rendimento, on='faixa_rendimento')
df_percentual_cotagio_faixa_rendimento['percentual_testes_positivos']= ((df_percentual_cotagio_faixa_rendimento['qtd_testes_positivos'] / df_percentual_cotagio_faixa_rendimento['qtd_testes_validos']) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_percentual_cotagio_faixa_rendimento.sort_values('percentual_testes_positivos', ascending=False),
    x='faixa_rendimento',
    y='percentual_testes_positivos', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por faixa de rendimento</b>',
    yaxis_title='%',
    showlegend=False,
    width=800,  
    height=600
)

fig.show()

**Qual é o valor médio do auxilio emergencial recebido entre infectados por faixa de rendimento?**

df_auxilio_emergencial_infectados = spark.sql(
    '''
        SELECT faixa_rendimento, sum(auxlio_emergencia_covid) as soma_auxlio_covid_faixa_rendimento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND auxlio_emergencia_covid != 'Não aplicável' AND auxlio_emergencia_covid != 'Não' AND faixa_rendimento != 'None'
        GROUP BY faixa_rendimento
    '''
).toPandas()

df_auxilio_emergencial_infectados['media_auxlio_covid_faixa_rendimento'] = (df_auxilio_emergencial_infectados.soma_auxlio_covid_faixa_rendimento / df_auxilio_emergencial_infectados.qtd_testes_positivos).round(decimals=2)

fig = px.bar(
    data_frame=df_auxilio_emergencial_infectados.sort_values('media_auxlio_covid_faixa_rendimento', ascending=False),
    x='faixa_rendimento',
    y='media_auxlio_covid_faixa_rendimento', 
    color='faixa_rendimento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'media_auxlio_covid_faixa_rendimento': 'Valor médio (R$)',
        'faixa_rendimento': 'Faixa de rendimento (R$)'
    }
)

fig.update_layout(
    title='<b>Valor médio do auxílio emergencial recebido por infectados por faixa de rendimento</b>',
    showlegend=False,
    width=800, 
    height=600
)

fig.update_xaxes(tickangle=45)

fig.show()

**Comparativo entre o mínimo do rendimento informado e o valor médio de auxílio emergencial recebido entre infectados por faixa de rendimento**

#### **<p style='color:gray'> Estatísticas descritivas de incidência da doença em relação as condições de trabalho</p>**

**Qual é a quantidade de infectados por tipo de trabalho?**

df_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()

fig = px.bar(
    data_frame=df_tipo_trabalho.sort_values('qtd_testes_positivos', ascending=False),
    x='qtd_testes_positivos',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado',
    color_discrete_sequence=['#8B0000'],
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_tipo_trabalho_realizado':'Tipo de trabalho realizado'
    }
)

fig.update_layout(
    title='<b>Quantidade de infectados por tipo de trabalho realizado</b>',
    showlegend=False,
    width=1200, 
    height=800
)

fig.show()

**Qual é a porcentagem de infectados por tipo de trabalho?**

df_qtd_testes_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()

df_qtd_testes_positivos_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()

df_tipo_trabalho = pd.merge(df_qtd_testes_tipo_trabalho, df_qtd_testes_positivos_tipo_trabalho, on='questao_tipo_trabalho_realizado')
df_tipo_trabalho['percentual_testes_positivos'] = ((df_tipo_trabalho['qtd_testes_positivos'] / df_tipo_trabalho['qtd_testes_validos']) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_tipo_trabalho.sort_values('percentual_testes_positivos', ascending=False),
    x='percentual_testes_positivos',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado',
    color_discrete_sequence=['#8B0000'],
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'questao_tipo_trabalho_realizado': 'Tipo de trabalho realizado'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por tipo de trabalho</b>',
    xaxis_title='%', 
    showlegend=False,
    width=1200, 
    height=800
)

fig.show()

**Qual é a porcentagem de infectados por motivo de afastamento do trabalho?**

df_percentual_motivo_afastamento = spark.sql(
    '''
        SELECT questao_motivo_afastamento, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_motivo_afastamento != 'Não aplicável'
        GROUP BY questao_motivo_afastamento
    '''
).toPandas()

df_percentual_motivo_afastamento['percentual_motivo_afastamento'] = ((df_percentual_tipo_teste.qtd_testes_positivos / sum(df_percentual_tipo_teste.qtd_testes_positivos)) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_percentual_motivo_afastamento.sort_values('percentual_motivo_afastamento', ascending=False), 
    x = 'questao_motivo_afastamento', 
    y = 'percentual_motivo_afastamento',
    color='questao_motivo_afastamento',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_motivo_afastamento': 'Percentual',
        'questao_motivo_afastamento': 'Motivo do afastamento'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por motivo do afastamento</b>',
    yaxis_title='%',
    legend_title='Legenda',
    xaxis_visible=False,
    xaxis_showticklabels=False,
    width=1200, 
    height=600
)

fig.show()

#### **<p style='color:gray'> Estatísticas decritivas dos métodos de testagem realizados</p>**

**Qual é a quantidade de testes por tipo de teste?**


df_tipo_teste_covid = spark.sql(
    '''
        SELECT tipo_teste, count(tipo_teste) AS qtd_teste
        FROM df_temp 
        WHERE tipo_teste != 'Não aplicável'
        GROUP BY tipo_teste
    '''
).toPandas()

fig = px.bar(
    data_frame=df_tipo_teste_covid.sort_values('qtd_teste', ascending=False), 
    x='tipo_teste',
    y='qtd_teste',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_teste': 'Quantidade',
        'tipo_teste': 'Tipo de teste'
    }
)

fig.update_traces(
    pull=[0, 0, 0.1]

)

fig.update_layout(
    title='<b>Quantidade de testes por tipo de teste</b>',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcentagem de infectados por tipo de teste?**


df_testes_validos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo'
        GROUP BY tipo_teste

    '''
).toPandas()

df_testes_positivos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY tipo_teste
    '''
).toPandas()

df_percentual_tipo_teste = pd.merge(df_testes_validos_tipo_teste, df_testes_positivos_tipo_teste, on='tipo_teste')
df_percentual_tipo_teste['percentual_testes_positivos'] = ((df_percentual_tipo_teste.qtd_testes_positivos / df_percentual_tipo_teste.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_percentual_tipo_teste.sort_values('percentual_testes_positivos', ascending=False), 
    x='tipo_teste',
    y='percentual_testes_positivos',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'tipo_teste': 'Tipo de teste'
    }
)

fig.update_traces(
    pull=[0, 0.1, 0]

)

fig.update_layout(
    title='<b>Porcentagem de infectados por tipo de teste</b>',
    legend_title='Legenda',
    yaxis_title='%',
    width=800, 
    height=600
)

fig.show()

**Qual é a porcentagem de testes inconclusivos por tipo de teste?**

df_testes_validos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' OR resultado_teste = 'Inconclusivo'
        GROUP BY tipo_teste

    '''
).toPandas()

df_testes_inconclusivos_tipo_teste = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_testes_inconclusivos
        FROM df_temp 
        WHERE resultado_teste = 'Inconclusivo'
        GROUP BY tipo_teste
    '''
).toPandas()

df_percentual_tipo_teste = pd.merge(df_testes_validos_tipo_teste, df_testes_inconclusivos_tipo_teste, on='tipo_teste')
df_percentual_tipo_teste['percentual_testes_inconclusivos'] = ((df_percentual_tipo_teste.qtd_testes_inconclusivos / df_percentual_tipo_teste.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_percentual_tipo_teste.sort_values('percentual_testes_inconclusivos', ascending=False), 
    x='tipo_teste',
    y='percentual_testes_inconclusivos',
    color='tipo_teste',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_inconclusivos': 'Percentual de testes',
        'tipo_teste': 'Tipo de teste'
    }
)

fig.update_traces(
    pull=[0, 0,1, 0]

)

fig.update_layout(
    title='<b>Porcentagem de testes com resultado inconclusivo por tipo de teste</b>',
    legend_title='Legenda',
    yaxis_title='%',
    width=800, 
    height=600
)

fig.show()